In [2]:
from datetime import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm as cm
%matplotlib inline
import scipy.interpolate
import pandas as pd

In [3]:
import shapefile, sys
import os
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.ticker import MaxNLocator
import pyproj

In [3]:
sf = shapefile.Reader(os.getcwd() + '/' + 'shape/tl_2013_us_zcta510.shp')

In [3]:
proj = pyproj.Proj(init="esri:26918")

In [4]:
import json

In [8]:
#read json as a string
json_file = open('shape/us_zip.json')
json_str = json_file.read()

In [9]:
#convert to dictionary
json_data = json.loads(json_str)

In [20]:
# json_data['features'][1000]

In [21]:
df = pd.read_csv('zip_borough.csv')

In [23]:
NY_zips = df['Incident Zip'].tolist()

In [18]:
#create dictionary for nyc
nyc_zips = {}
nyc_zips['type'] = u'FeatureCollection'
nyc_zips['features'] = []

In [35]:
#loop through nys dictionary and keep entries where city is New York
for i in json_data['features']:
    if int(i['properties']['ZCTA5CE10']) in NY_zips:
        nyc_zips['features'].append(i)

In [41]:
# nyc_zips['features'][0]

In [114]:
with open('nyc_zips_n.json', 'w') as outfile:
    json.dump(nyc, outfile)

In [5]:
with open('nyc_zips_n.json') as f:
    nyc = json.load(f)

In [88]:
l = []
for e in nyc['features']:
    l.append(np.vstack([np.array(i).ravel().reshape(-1, 2) for i in e['geometry']['coordinates']]))

In [112]:
# l[7]

In [8]:
# nyc['features'][7]['geometry']

In [101]:
for e in nyc['features']:
    bbox = []
    x = np.vstack([np.array(i).ravel().reshape(-1, 2) for i in e['geometry']['coordinates']])
    # get min lng and min lat
    minx = np.min(x,axis=0)
    # get max lng and max lat
    maxx = np.max(x,axis=0)
    bbox = np.r_[minx,maxx].tolist()
    e['geometry'][u'bbox'] = bbox

In [9]:
from rtree import index

In [10]:
idx = index.Index()

In [11]:
for i in nyc['features']:
    bbox = i['geometry']['bbox']
    zipc = i['properties']['ZCTA5CE10']
    idx.insert(int(zipc),tuple(bbox))

In [12]:
hits = list(idx.intersection((-73.986493, 40.693264)))
hits

[11201]

In [66]:
x = np.array(nyc['features'][7]['geometry']['coordinates'])

In [74]:
x[1][0]

[[-73.939632, 40.778954],
 [-73.939508, 40.779224],
 [-73.939572, 40.779368],
 [-73.939537, 40.779694],
 [-73.939308, 40.779851],
 [-73.939336, 40.780083],
 [-73.938973, 40.780493],
 [-73.938803, 40.780602],
 [-73.938829, 40.780763],
 [-73.938101, 40.781237],
 [-73.937515, 40.781133],
 [-73.937492, 40.781077],
 [-73.9374, 40.781039],
 [-73.937328, 40.780985],
 [-73.937602, 40.780592],
 [-73.938833, 40.780059],
 [-73.938627, 40.779402],
 [-73.939217, 40.778565],
 [-73.939632, 40.778954]]

In [16]:
x = np.array(nyc['features'][0]['geometry']['coordinates'][0])

In [17]:
x.shape

(57, 2)

In [21]:
minx = np.min(x,axis=0)

In [26]:
minx

array([-73.972553,  40.759284])

In [27]:
maxx = np.max(x,axis=0)

In [28]:
maxx

array([-73.952308,  40.770517])

In [33]:
xx = np.r_[minx,maxx]

In [52]:
xx = xx.tolist()

In [54]:
yy = tuple(xx)

In [55]:
yy

(-73.972553, 40.759284, -73.952308, 40.770517)

In [14]:
import cPickle as pickle

In [15]:
pickle.dump(idx, open( "zip_tree.p", "wb" ))